# Text Converter

## Text to Binary

In [ ]:
def text_to_binary(text):
    # Mengubah setiap karakter menjadi representasi biner 8-bit
    binary_representation = ''.join(format(ord(char), '08b') for char in text)
    return binary_representation

In [ ]:
plaintext = "Komputer"
plaintext_binary = text_to_binary(plaintext)
print(f"Plaintext: {plaintext_binary}")

key = "4611422062"
key_binary = text_to_binary(key)
print(f"Key: {key_binary}")

Plaintext: 0100101101101111011011010111000001110101011101000110010101110010
Key: 00110100001101100011000100110001001101000011001000110010001100000011011000110010


## Binary to Text

In [22]:
def binary_to_text(binary_string):
    # Membagi biner menjadi grup 8 bit
    chars = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]

    # Mengubah setiap grup biner menjadi karakter
    text = ''.join([chr(int(char, 2)) for char in chars])

    return text

In [ ]:
input = key_binary
text = binary_to_text(input)
print(text)

4611422062


## Binary to Hex

In [ ]:
def binary_to_hex(binary_string):
    # Pastikan panjang string biner adalah kelipatan 4
    binary_string = binary_string.zfill(len(binary_string) + (4 - len(binary_string) % 4) % 4)

    # Konversi biner ke integer, lalu integer ke heksadesimal
    hex_string = hex(int(binary_string, 2))[2:]  # [2:] untuk menghilangkan prefix '0x'

    return hex_string.upper()

In [ ]:
encrypt_result = '1101001001100101101100010100011011000010011111001010101011010011'
hex_result = binary_to_hex(encrypt_result)
print(hex_result)

D265B146C27CAAD3


# DES Base

## Table

In [67]:
# Define the necessary DES tables (initial permutation, final permutation, etc.)
INITIAL_PERMUTATION_TABLE = [58, 50, 42, 34, 26, 18, 10, 2,
                             60, 52, 44, 36, 28, 20, 12, 4,
                             62, 54, 46, 38, 30, 22, 14, 6,
                             64, 56, 48, 40, 32, 24, 16, 8,
                             57, 49, 41, 33, 25, 17, 9, 1,
                             59, 51, 43, 35, 27, 19, 11, 3,
                             61, 53, 45, 37, 29, 21, 13, 5,
                             63, 55, 47, 39, 31, 23, 15, 7]

FINAL_PERMUTATION_TABLE = [40, 8, 48, 16, 56, 24, 64, 32,
                           39, 7, 47, 15, 55, 23, 63, 31,
                           38, 6, 46, 14, 54, 22, 62, 30,
                           37, 5, 45, 13, 53, 21, 61, 29,
                           36, 4, 44, 12, 52, 20, 60, 28,
                           35, 3, 43, 11, 51, 19, 59, 27,
                           34, 2, 42, 10, 50, 18, 58, 26,
                           33, 1, 41, 9, 49, 17, 57, 25]

EXPANSION_TABLE = [32, 1, 2, 3, 4, 5,
                   4, 5, 6, 7, 8, 9,
                   8, 9, 10, 11, 12, 13,
                   12, 13, 14, 15, 16, 17,
                   16, 17, 18, 19, 20, 21,
                   20, 21, 22, 23, 24, 25,
                   24, 25, 26, 27, 28, 29,
                   28, 29, 30, 31, 32, 1]

# S-boxes for DES (there are 8 S-boxes)
S_BOXES = [
    # S1
    [[14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
     [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
     [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
     [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]],

    # S2
    [[15, 1, 8, 14, 6, 11, 3, 4, 9, 7, 2, 13, 12, 0, 5, 10],
     [3, 13, 4, 7, 15, 2, 8, 14, 12, 0, 1, 10, 6, 9, 11, 5],
     [0, 14, 7, 11, 10, 4, 13, 1, 5, 8, 12, 6, 9, 3, 2, 15],
     [13, 8, 10, 1, 3, 15, 4, 2, 11, 6, 7, 12, 0, 5, 14, 9]],

    # S3
    [[10, 0, 9, 14, 6, 3, 15, 5, 1, 13, 12, 7, 11, 4, 2, 8],
     [13, 7, 0, 9, 3, 4, 6, 10, 2, 8, 5, 14, 12, 11, 15, 1],
     [13, 6, 4, 9, 8, 15, 3, 0, 11, 1, 2, 12, 5, 10, 14, 7],
     [1, 10, 13, 0, 6, 9, 8, 7, 4, 15, 14, 3, 11, 5, 2, 12]],

    # S4
    [[7, 13, 14, 3, 0, 6, 9, 10, 1, 2, 8, 5, 11, 12, 4, 15],
     [13, 8, 11, 5, 6, 15, 0, 3, 4, 7, 2, 12, 1, 10, 14, 9],
     [10, 6, 9, 0, 12, 11, 7, 13, 15, 1, 3, 14, 5, 2, 8, 4],
     [3, 15, 0, 6, 10, 1, 13, 8, 9, 4, 5, 11, 12, 7, 2, 14]],

    # S5
    [[2, 12, 4, 1, 7, 10, 11, 6, 8, 5, 3, 15, 13, 0, 14, 9],
     [14, 11, 2, 12, 4, 7, 13, 1, 5, 0, 15, 10, 3, 9, 8, 6],
     [4, 2, 1, 11, 10, 13, 7, 8, 15, 9, 12, 5, 6, 3, 0, 14],
     [11, 8, 12, 7, 1, 14, 2, 13, 6, 15, 0, 9, 10, 4, 5, 3]],

    # S6
    [[12, 1, 10, 15, 9, 2, 6, 8, 0, 13, 3, 4, 14, 7, 5, 11],
     [10, 15, 4, 2, 7, 12, 9, 5, 6, 1, 13, 14, 0, 11, 3, 8],
     [9, 14, 15, 5, 2, 8, 12, 3, 7, 0, 4, 10, 1, 13, 11, 6],
     [4, 3, 2, 12, 9, 5, 15, 10, 11, 14, 1, 7, 6, 0, 8, 13]],

    # S7
    [[4, 11, 2, 14, 15, 0, 8, 13, 3, 12, 9, 7, 5, 10, 6, 1],
     [13, 0, 11, 7, 4, 9, 1, 10, 14, 3, 5, 12, 2, 15, 8, 6],
     [1, 4, 11, 13, 12, 3, 7, 14, 10, 15, 6, 8, 0, 5, 9, 2],
     [6, 11, 13, 8, 1, 4, 10, 7, 9, 5, 0, 15, 14, 2, 3, 12]],

    # S8
    [[13, 2, 8, 4, 6, 15, 11, 1, 10, 9, 3, 14, 5, 0, 12, 7],
     [1, 15, 13, 8, 10, 3, 7, 4, 12, 5, 6, 11, 0, 14, 9, 2],
     [7, 11, 4, 1, 9, 12, 14, 2, 0, 6, 10, 13, 15, 3, 5, 8],
     [2, 1, 14, 7, 4, 10, 8, 13, 15, 12, 9, 0, 3, 5, 6, 11]]
]


# Permutation table for function f output
PERMUTATION_TABLE = [16, 7, 20, 21, 29, 12, 28, 17,
                     1, 15, 23, 26, 5, 18, 31, 10,
                     2, 8, 24, 14, 32, 27, 3, 9,
                     19, 13, 30, 6, 22, 11, 4, 25]

# Permutation Choice 1 table for the key (PC-1)
PC_1 = [57, 49, 41, 33, 25, 17, 9,
        1, 58, 50, 42, 34, 26, 18,
        10, 2, 59, 51, 43, 35, 27,
        19, 11, 3, 60, 52, 44, 36,
        63, 55, 47, 39, 31, 23, 15,
        7, 62, 54, 46, 38, 30, 22,
        14, 6, 61, 53, 45, 37, 29,
        21, 13, 5, 28, 20, 12, 4]

# Key shift schedule (1 or 2 bit shifts)
SHIFT_SCHEDULE = [1, 1, 2, 2, 2, 2, 2, 2,
                  1, 2, 2, 2, 2, 2, 2, 1]

# Permutation Choice 2 table for the key (PC-2)
PC_2 = [14, 17, 11, 24, 1, 5,
        3, 28, 15, 6, 21, 10,
        23, 19, 12, 4, 26, 8,
        16, 7, 27, 20, 13, 2,
        41, 52, 31, 37, 47, 55,
        30, 40, 51, 45, 33, 48,
        44, 49, 39, 56, 34, 53,
        46, 42, 50, 36, 29, 32]

## Program

In [68]:
# Helper functions
def pad_block(block, block_size=64):
    """Pads block to the required block size using zero-padding."""
    if len(block) < block_size:
        block += '0' * (block_size - len(block))
    return block

def permute(block, table):
    """Permute a block based on a table (indexes)."""
    # Ensure block is 64 bits
    block = pad_block(block, max(table))
    return ''.join(block[i - 1] for i in table)

def xor(a, b):
    """Perform XOR between two binary strings."""
    return ''.join('0' if i == j else '1' for i, j in zip(a, b))

def s_box_lookup(bits, sbox_number):
    """S-box substitution."""
    sbox = S_BOXES[sbox_number]
    row = int(bits[0] + bits[5], 2)  # Row from first and last bit
    col = int(bits[1:5], 2)  # Column from the middle 4 bits
    return format(sbox[row][col], '04b')

def split_half(block):
    """Split block into two halves."""
    mid = len(block) // 2
    return block[:mid], block[mid:]

# Key scheduling for 16 rounds of subkeys
def key_schedule(key):
    key = permute(key, PC_1)
    C, D = split_half(key)
    subkeys = []
    for i in range(16):
        C = C[SHIFT_SCHEDULE[i]:] + C[:SHIFT_SCHEDULE[i]]
        D = D[SHIFT_SCHEDULE[i]:] + D[:SHIFT_SCHEDULE[i]]
        subkey = permute(C + D, PC_2)
        subkeys.append(subkey)
    return subkeys

# DES Function f (used in each round)
def function_f(R, subkey):
    expanded_R = permute(R, EXPANSION_TABLE)  # Expand R from 32 to 48 bits
    xored = xor(expanded_R, subkey)  # XOR expanded R with subkey

    # Apply S-box substitution
    sbox_output = ''.join(s_box_lookup(xored[i*6:(i+1)*6], i) for i in range(8))

    # Apply permutation after S-box
    return permute(sbox_output, PERMUTATION_TABLE)

## Enkripsi

In [69]:
def des_encrypt(plaintext, key):
    # Initial permutation
    plaintext = permute(plaintext, INITIAL_PERMUTATION_TABLE)
    L, R = split_half(plaintext)
    subkeys = key_schedule(key)

    # 16 rounds of DES
    for i in range(16):
        L, R = R, xor(L, function_f(R, subkeys[i]))

    # Final permutation
    return permute(R + L, FINAL_PERMUTATION_TABLE)

## Dekripsi

In [70]:
def des_decrypt(ciphertext, key):
    # Initial permutation
    ciphertext = permute(ciphertext, INITIAL_PERMUTATION_TABLE)
    L, R = split_half(ciphertext)
    subkeys = key_schedule(key)

    # 16 rounds of DES (in reverse order for decryption)
    for i in range(15, -1, -1):
        L, R = R, xor(L, function_f(R, subkeys[i]))

    # Final permutation
    return permute(R + L, FINAL_PERMUTATION_TABLE)

# DES Image

## Converter

In [71]:
from PIL import Image
import numpy as np

# Fungsi untuk membuka dan mengonversi gambar menjadi byte data
def load_image(image_path):
    img = Image.open(image_path)
    img_data = np.array(img)  # Konversi gambar ke array numpy
    return img, img_data

# Fungsi untuk menyimpan gambar yang sudah diacak setelah enkripsi
def save_image(image_data, output_path):
    img = Image.fromarray(image_data.astype('uint8'))  # Konversi array numpy kembali ke gambar
    img.save(output_path)

# Fungsi untuk memecah gambar menjadi blok-blok 64-bit
def image_to_blocks(image_data):
    flat_image = image_data.flatten()  # Membuat array datar dari gambar (1D)
    binary_data = ''.join([format(pixel, '08b') for pixel in flat_image])  # Konversi piksel menjadi biner
    return [binary_data[i:i+64] for i in range(0, len(binary_data), 64)]  # Memecah menjadi blok 64-bit

# Fungsi untuk menggabungkan blok-blok yang telah dienkripsi kembali menjadi gambar
def blocks_to_image(blocks, original_shape):
    binary_string = ''.join(blocks)
    byte_array = [int(binary_string[i:i+8], 2) for i in range(0, len(binary_string), 8)]
    return np.array(byte_array).reshape(original_shape)  # Mengembalikan ke bentuk gambar asli

## Enkripsi

In [72]:
def encrypt_image(input_image_path, key, output_image_path):
    # Open the image and convert to RGB or Grayscale
    img = Image.open(input_image_path)

    # Check if the image is grayscale or color
    is_grayscale = img.mode == 'L'
    if is_grayscale:
        img = img.convert('L')  # Convert to grayscale if it's not already

    pixels = list(img.getdata())

    # Convert pixels to binary
    if is_grayscale:
        binary_pixels = ''.join(format(pixel, '08b') for pixel in pixels)
    else:
        binary_pixels = ''.join(format(component, '08b') for pixel in pixels for component in pixel)

    encrypted_data = ''

    # Encrypt in 64-bit blocks
    for i in range(0, len(binary_pixels), 64):
        block = binary_pixels[i:i + 64]
        block = pad_block(block)  # Pad block if needed
        encrypted_block = des_encrypt(block, key)
        encrypted_data += encrypted_block

    # Save encrypted data into an image
    if is_grayscale:
        encrypted_image = Image.new('L', img.size)  # Create a new grayscale image
        encrypted_pixels = []

        for i in range(0, len(encrypted_data), 8):  # 8 bits for grayscale
            gray_value = int(encrypted_data[i:i + 8], 2)
            encrypted_pixels.append(gray_value)

        encrypted_image.putdata(encrypted_pixels)
    else:
        encrypted_image = Image.new('RGB', img.size)  # Create a new color image
        encrypted_pixels = []

        for i in range(0, len(encrypted_data), 24):  # 3 * 8 = 24 bits for RGB
            r = int(encrypted_data[i:i + 8], 2)
            g = int(encrypted_data[i + 8:i + 16], 2)
            b = int(encrypted_data[i + 16:i + 24], 2)
            encrypted_pixels.append((r, g, b))

        encrypted_image.putdata(encrypted_pixels)

    encrypted_image.save(output_image_path)

## Dekripsi

In [73]:
def decrypt_image(encrypted_image_path, key, output_image_path):
    # Open the image and convert to RGB or Grayscale
    img = Image.open(encrypted_image_path)

    # Check if the image is grayscale or color
    is_grayscale = img.mode == 'L'
    if is_grayscale:
        img = img.convert('L')  # Convert to grayscale if it's not already

    pixels = list(img.getdata())

    # Convert pixels to binary
    if is_grayscale:
        binary_pixels = ''.join(format(pixel, '08b') for pixel in pixels)
    else:
        binary_pixels = ''.join(format(component, '08b') for pixel in pixels for component in pixel)

    decrypted_data = ''

    # Decrypt in 64-bit blocks
    for i in range(0, len(binary_pixels), 64):
        block = binary_pixels[i:i + 64]
        decrypted_block = des_decrypt(block, key)
        decrypted_data += decrypted_block

    # Save decrypted data into an image
    if is_grayscale:
        decrypted_image = Image.new('L', img.size)  # Create a new grayscale image
        decrypted_pixels = []

        for i in range(0, len(decrypted_data), 8):  # 8 bits for grayscale
            gray_value = int(decrypted_data[i:i + 8], 2)
            decrypted_pixels.append(gray_value)

        decrypted_image.putdata(decrypted_pixels)
    else:
        decrypted_image = Image.new('RGB', img.size)  # Create a new color image
        decrypted_pixels = []

        for i in range(0, len(decrypted_data), 24):  # 3 * 8 = 24 bits for RGB
            r = int(decrypted_data[i:i + 8], 2)
            g = int(decrypted_data[i + 8:i + 16], 2)
            b = int(decrypted_data[i + 16:i + 24], 2)
            decrypted_pixels.append((r, g, b))

        decrypted_image.putdata(decrypted_pixels)

    decrypted_image.save(output_image_path)

## Testing

In [74]:
# Path untuk gambar input dan output
input_image_path = 'lena.jpg'
encrypted_image_path = 'lena_encrypted.jpg'
decrypted_image_path = 'lena_decrypted.jpg'

# Kunci DES (harus berupa 64-bit atau 8 karakter)
key = '46114220'

In [75]:
# Enkripsi gambar
encrypt_image(input_image_path, key, encrypted_image_path)
print(f"Gambar telah terenkripsi dan disimpan di {encrypted_image_path}")

Gambar telah terenkripsi dan disimpan di lena_encrypted.jpg


In [76]:
# Dekripsi gambar
decrypt_image(encrypted_image_path, key, decrypted_image_path)
print(f"Gambar telah terdekripsi dan disimpan di {decrypted_image_path}")

Gambar telah terdekripsi dan disimpan di lena_decrypted.jpg


# DES Audio